# Day 09. Exercise 00
# Regularization

## 0. Imports

In [1]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import tqdm
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold
import joblib

## 1. Preprocessing

1. Read the file `dayofweek.csv` that you used in the previous day to a dataframe.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [2]:
df = pd.read_csv("../data/dayofweek.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,numTrials,hour,dayofweek,user_0,user_1,user_10,user_11,user_12,user_13,user_14,...,lab02,lab03,lab03s,lab05s,laba04,laba04s,laba05,laba06,laba06s,project1
0,-0.788667,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.756764,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.724861,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.692958,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.661055,-2.562352,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,-0.533442,0.945382,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1682,-0.629151,0.945382,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1683,-0.597248,0.945382,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1684,-0.565345,0.945382,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('dayofweek', axis=1), df['dayofweek'], test_size=0.2, random_state=21, stratify=df['dayofweek'])

## 2. Logreg regularization

### a. Default regularization

1. Train a baseline model with the only parameters `random_state=21`, `fit_intercept=False`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model


The result of the code where you trained and evaluated the baseline model should be exactly like this (use `%%time` to get the info about how long it took to run the cell):

```
train -  0.62902   |   valid -  0.59259
train -  0.64633   |   valid -  0.62963
train -  0.63479   |   valid -  0.56296
train -  0.65622   |   valid -  0.61481
train -  0.63397   |   valid -  0.57778
train -  0.64056   |   valid -  0.59259
train -  0.64138   |   valid -  0.65926
train -  0.65952   |   valid -  0.56296
train -  0.64333   |   valid -  0.59701
train -  0.63674   |   valid -  0.62687
Average accuracy on crossval is 0.60165
Std is 0.02943
```

In [4]:
def crossval(X, y, cv, model):
    kf = StratifiedKFold(n_splits=cv)
    valids = []
    for train, test in kf.split(X, y):
        X_train , X_test = X.iloc[train], X.iloc[test]
        y_train , y_test = y.iloc[train] , y.iloc[test]
        model.fit(X_train, y_train)
        acc_valid = round(accuracy_score(y_test, model.predict(X_test)), 5)
        valids.append(acc_valid)
        acc_valid = format(acc_valid, '0.5f')
        acc_train = format(round(accuracy_score(y_train, model.predict(X_train)), 5), '0.5f')
        print(f"train - {acc_train} | valid - {acc_valid}")
        
    Avg_ = round(np.mean(valids), 5)
    std_ = round(np.std(valids), 5)
    print(f"Average accuracy on crossval is {Avg_}\nStd is {std_}")
        
    

In [5]:
logreg = LogisticRegression(fit_intercept=False, random_state=21)
crossval(X_train, y_train, 10, logreg)

train - 0.62902 | valid - 0.59259
train - 0.64633 | valid - 0.62963
train - 0.63479 | valid - 0.56296
train - 0.65622 | valid - 0.61481
train - 0.63397 | valid - 0.57778
train - 0.64056 | valid - 0.59259
train - 0.64138 | valid - 0.65926
train - 0.65952 | valid - 0.56296
train - 0.64333 | valid - 0.59701
train - 0.63674 | valid - 0.62687
Average accuracy on crossval is 0.60165
Std is 0.02943


### b. Optimizing regularization parameters

1. In the cells below try different values of penalty: `none`, `l1`, `l2` – you can change the values of solver too.

In [6]:
%%time
logreg = LogisticRegression(penalty='l2', fit_intercept=False, random_state=21)
crossval(X_train, y_train, 10, logreg)

train - 0.62902 | valid - 0.59259
train - 0.64633 | valid - 0.62963
train - 0.63479 | valid - 0.56296
train - 0.65622 | valid - 0.61481
train - 0.63397 | valid - 0.57778
train - 0.64056 | valid - 0.59259
train - 0.64138 | valid - 0.65926
train - 0.65952 | valid - 0.56296
train - 0.64333 | valid - 0.59701
train - 0.63674 | valid - 0.62687
Average accuracy on crossval is 0.60165
Std is 0.02943
CPU times: user 2.03 s, sys: 363 ms, total: 2.39 s
Wall time: 406 ms


In [7]:
%%time
logreg = LogisticRegression(penalty='l2', fit_intercept=False, random_state=21, solver='liblinear')
crossval(X_train, y_train, 10, logreg)

train - 0.61006 | valid - 0.56296
train - 0.61665 | valid - 0.61481
train - 0.61336 | valid - 0.59259
train - 0.62902 | valid - 0.60741
train - 0.60923 | valid - 0.55556
train - 0.61500 | valid - 0.57778
train - 0.61665 | valid - 0.61481
train - 0.64056 | valid - 0.53333
train - 0.62109 | valid - 0.58209
train - 0.61120 | valid - 0.57463
Average accuracy on crossval is 0.5816
Std is 0.02532
CPU times: user 609 ms, sys: 152 ms, total: 761 ms
Wall time: 179 ms


In [8]:
%%time
logreg = LogisticRegression(penalty='l1', fit_intercept=False, random_state=21, solver='liblinear')
crossval(X_train, y_train, 10, logreg)

train - 0.61830 | valid - 0.54815
train - 0.62737 | valid - 0.62222
train - 0.60511 | valid - 0.54074
train - 0.63644 | valid - 0.62222
train - 0.62407 | valid - 0.55556
train - 0.62325 | valid - 0.58519
train - 0.61253 | valid - 0.63704
train - 0.64716 | valid - 0.58519
train - 0.63015 | valid - 0.59701
train - 0.61367 | valid - 0.59701
Average accuracy on crossval is 0.58903
Std is 0.03129
CPU times: user 829 ms, sys: 172 ms, total: 1 s
Wall time: 170 ms


In [9]:
%%time
logreg = LogisticRegression(penalty='none', fit_intercept=False, random_state=21, solver='newton-cg')
crossval(X_train, y_train, 10, logreg)

train - 0.66694 | valid - 0.63704
train - 0.65787 | valid - 0.65926
train - 0.66694 | valid - 0.57778
train - 0.66529 | valid - 0.62963
train - 0.66694 | valid - 0.62222
train - 0.65952 | valid - 0.57778
train - 0.65045 | valid - 0.69630
train - 0.68425 | valid - 0.61481
train - 0.66474 | valid - 0.62687
train - 0.65651 | valid - 0.60448
Average accuracy on crossval is 0.62462
Std is 0.03379
CPU times: user 10.1 s, sys: 1.23 s, total: 11.4 s
Wall time: 1.92 s


## 3. SVM regularization

### a. Default regularization

1. Train a baseline model with the only parameters `probability=True`, `kernel='linear'`, `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [10]:
my_svm = svm.SVC(random_state = 21, probability=True, kernel='linear')
crossval(X_train, y_train, 10, my_svm)

train - 0.70486 | valid - 0.65926
train - 0.69662 | valid - 0.75556
train - 0.69415 | valid - 0.62222
train - 0.70239 | valid - 0.65185
train - 0.69085 | valid - 0.65185
train - 0.68920 | valid - 0.64444
train - 0.69250 | valid - 0.72593
train - 0.70074 | valid - 0.62222
train - 0.69605 | valid - 0.61940
train - 0.71087 | valid - 0.63433
Average accuracy on crossval is 0.65871
Std is 0.04359


### b. Optimizing regularization parameters

1. In the cells below try different values of the parameter `C`.

In [11]:
my_svm = svm.SVC(C=2.0, random_state = 21, probability=True, kernel='linear')
crossval(X_train, y_train, 10, my_svm)

train - 0.70734 | valid - 0.65926
train - 0.71393 | valid - 0.75556
train - 0.74526 | valid - 0.63704
train - 0.71558 | valid - 0.66667
train - 0.71146 | valid - 0.67407
train - 0.70157 | valid - 0.63704
train - 0.70651 | valid - 0.71852
train - 0.70981 | valid - 0.64444
train - 0.72405 | valid - 0.64925
train - 0.72488 | valid - 0.64179
Average accuracy on crossval is 0.66836
Std is 0.03721


In [12]:
my_svm = svm.SVC(C=5.0, random_state = 21, probability=True, kernel='linear')
crossval(X_train, y_train, 10, my_svm)

train - 0.71723 | valid - 0.65185
train - 0.76257 | valid - 0.81481
train - 0.77906 | valid - 0.68148
train - 0.73537 | valid - 0.67407
train - 0.72135 | valid - 0.73333
train - 0.72960 | valid - 0.70370
train - 0.72712 | valid - 0.71852
train - 0.75021 | valid - 0.65926
train - 0.76112 | valid - 0.67164
train - 0.76359 | valid - 0.68657
Average accuracy on crossval is 0.69952
Std is 0.04541


In [13]:
my_svm = svm.SVC(C=100.0, random_state = 21, probability=True, kernel='linear')
crossval(X_train, y_train, 10, my_svm)

train - 0.78401 | valid - 0.74815
train - 0.79720 | valid - 0.84444
train - 0.80956 | valid - 0.72593
train - 0.79060 | valid - 0.76296
train - 0.79060 | valid - 0.77778
train - 0.79637 | valid - 0.74815
train - 0.78401 | valid - 0.77037
train - 0.80462 | valid - 0.73333
train - 0.79819 | valid - 0.70896
train - 0.79901 | valid - 0.73881
Average accuracy on crossval is 0.75589
Std is 0.0355


## 4. Tree

### a. Default regularization

1. Train a baseline model with the only parameter `max_depth=10` and `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [14]:
dtc_model = DecisionTreeClassifier(max_depth = 10, random_state = 21)
crossval(X_train, y_train, 10, dtc_model)

train - 0.81039 | valid - 0.74074
train - 0.77741 | valid - 0.74074
train - 0.83347 | valid - 0.70370
train - 0.79720 | valid - 0.76296
train - 0.82440 | valid - 0.75556
train - 0.80379 | valid - 0.68889
train - 0.80709 | valid - 0.76296
train - 0.80132 | valid - 0.65926
train - 0.80807 | valid - 0.75373
train - 0.80478 | valid - 0.68657
Average accuracy on crossval is 0.72551
Std is 0.03562


### b. Optimizing regularization parameters

1. In the cells below try different values of the parameter `max_depth`.
2. As a bonus, play with other regularization parameters trying to find the best combination.

In [15]:
dtc_model = DecisionTreeClassifier(max_depth = 20, random_state = 21)
crossval(X_train, y_train, 10, dtc_model)

train - 0.98846 | valid - 0.86667
train - 0.99011 | valid - 0.91111
train - 0.98681 | valid - 0.85926
train - 0.98763 | valid - 0.91111
train - 0.98928 | valid - 0.88148
train - 0.98186 | valid - 0.85926
train - 0.98846 | valid - 0.91852
train - 0.99176 | valid - 0.89630
train - 0.99094 | valid - 0.88060
train - 0.98847 | valid - 0.88060
Average accuracy on crossval is 0.88649
Std is 0.02075


In [16]:
dtc_model = DecisionTreeClassifier(max_depth = 23, random_state = 21)
crossval(X_train, y_train, 10, dtc_model)

train - 0.99753 | valid - 0.85926
train - 0.99588 | valid - 0.90370
train - 0.99588 | valid - 0.86667
train - 0.99753 | valid - 0.89630
train - 0.99670 | valid - 0.88148
train - 0.99423 | valid - 0.85926
train - 0.99505 | valid - 0.92593
train - 0.99670 | valid - 0.88148
train - 0.99918 | valid - 0.88060
train - 0.99506 | valid - 0.88060
Average accuracy on crossval is 0.88353
Std is 0.01964


In [17]:
dtc_model = DecisionTreeClassifier(max_depth = 21, random_state = 21)
crossval(X_train, y_train, 10, dtc_model)

train - 0.99176 | valid - 0.86667
train - 0.99176 | valid - 0.90370
train - 0.99093 | valid - 0.85926
train - 0.99176 | valid - 0.89630
train - 0.99093 | valid - 0.88148
train - 0.98516 | valid - 0.88148
train - 0.98928 | valid - 0.91852
train - 0.99423 | valid - 0.87407
train - 0.99588 | valid - 0.86567
train - 0.99012 | valid - 0.88060
Average accuracy on crossval is 0.88278
Std is 0.01757


In [18]:
dtc_model = DecisionTreeClassifier(max_depth = 100, random_state = 21)
crossval(X_train, y_train, 10, dtc_model)

train - 1.00000 | valid - 0.85926
train - 1.00000 | valid - 0.91852
train - 1.00000 | valid - 0.86667
train - 1.00000 | valid - 0.91111
train - 1.00000 | valid - 0.88148
train - 1.00000 | valid - 0.85185
train - 1.00000 | valid - 0.92593
train - 1.00000 | valid - 0.88148
train - 1.00000 | valid - 0.88060
train - 1.00000 | valid - 0.88060
Average accuracy on crossval is 0.88575
Std is 0.02374


## 5. Random forest

### a. Default regularization

1. Train a baseline model with the only parameters `n_estimators=50`, `max_depth=14`, `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [19]:
rf = RandomForestClassifier(n_estimators = 50, max_depth = 14, random_state = 21)
crossval(X_train, y_train, 10, rf)

train - 0.96455 | valid - 0.88148
train - 0.96208 | valid - 0.91852
train - 0.96785 | valid - 0.86667
train - 0.96455 | valid - 0.89630
train - 0.96538 | valid - 0.91111
train - 0.96538 | valid - 0.88148
train - 0.97115 | valid - 0.91852
train - 0.96867 | valid - 0.85185
train - 0.97364 | valid - 0.88060
train - 0.97941 | valid - 0.86567
Average accuracy on crossval is 0.88722
Std is 0.02205


### b. Optimizing regularization parameters

1. In the new cells try different values of the parameters `max_depth` and `n_estimators`.
2. As a bonus, play with other regularization parameters trying to find the best combination.

In [20]:
rf = RandomForestClassifier(n_estimators = 100, max_depth = 14, random_state = 21)
crossval(X_train, y_train, 10, rf)

train - 0.96950 | valid - 0.87407
train - 0.96867 | valid - 0.91111
train - 0.97197 | valid - 0.87407
train - 0.96867 | valid - 0.89630
train - 0.96867 | valid - 0.90370
train - 0.96702 | valid - 0.85926
train - 0.97444 | valid - 0.91111
train - 0.96950 | valid - 0.88148
train - 0.97117 | valid - 0.88060
train - 0.97199 | valid - 0.86567
Average accuracy on crossval is 0.88574
Std is 0.01773


In [21]:
rf = RandomForestClassifier(n_estimators = 50, max_depth = 30, random_state = 21)
crossval(X_train, y_train, 10, rf)

train - 1.00000 | valid - 0.90370
train - 1.00000 | valid - 0.94074
train - 1.00000 | valid - 0.89630
train - 1.00000 | valid - 0.93333
train - 1.00000 | valid - 0.91852
train - 0.99918 | valid - 0.89630
train - 1.00000 | valid - 0.91852
train - 1.00000 | valid - 0.89630
train - 1.00000 | valid - 0.92537
train - 0.99918 | valid - 0.88806
Average accuracy on crossval is 0.91171
Std is 0.0171


In [22]:
rf = RandomForestClassifier(n_estimators = 50, max_depth = 50, random_state = 21)
crossval(X_train, y_train, 10, rf)

train - 1.00000 | valid - 0.89630
train - 1.00000 | valid - 0.94815
train - 1.00000 | valid - 0.90370
train - 1.00000 | valid - 0.93333
train - 1.00000 | valid - 0.91111
train - 1.00000 | valid - 0.89630
train - 1.00000 | valid - 0.91852
train - 1.00000 | valid - 0.90370
train - 1.00000 | valid - 0.93284
train - 0.99918 | valid - 0.89552
Average accuracy on crossval is 0.91395
Std is 0.01762


In [23]:
rf = RandomForestClassifier(n_estimators = 30, max_depth = 70, random_state = 21)
crossval(X_train, y_train, 10, rf)

train - 1.00000 | valid - 0.88889
train - 1.00000 | valid - 0.94074
train - 0.99918 | valid - 0.91852
train - 1.00000 | valid - 0.94074
train - 1.00000 | valid - 0.90370
train - 1.00000 | valid - 0.89630
train - 1.00000 | valid - 0.92593
train - 1.00000 | valid - 0.88889
train - 1.00000 | valid - 0.92537
train - 1.00000 | valid - 0.90299
Average accuracy on crossval is 0.91321
Std is 0.01875


In [24]:
rf = RandomForestClassifier(n_estimators = 50, max_depth = 50, random_state = 21, criterion='entropy')
crossval(X_train, y_train, 10, rf)

train - 1.00000 | valid - 0.90370
train - 1.00000 | valid - 0.95556
train - 1.00000 | valid - 0.91852
train - 1.00000 | valid - 0.93333
train - 1.00000 | valid - 0.91111
train - 1.00000 | valid - 0.89630
train - 1.00000 | valid - 0.91852
train - 1.00000 | valid - 0.91111
train - 1.00000 | valid - 0.93284
train - 0.99918 | valid - 0.86567
Average accuracy on crossval is 0.91467
Std is 0.02294


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.
3. Analyze: for which weekday your model makes the most errors (in % of the total number of samples of that class in your test dataset).
4. Save the model.

In [25]:
best_model = RandomForestClassifier(n_estimators = 50, max_depth = 50, random_state = 21)

In [26]:
best_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=50, n_estimators=50, random_state=21)

In [27]:
accuracy_score(y_test, best_model.predict(X_test))

0.9289940828402367

In [28]:
predict = best_model.predict(X_test)

In [30]:
res = pd.DataFrame()
res['test'] = y_test
res.reset_index(inplace=True)
res['predict'] = pd.Series(predict)
res['diff'] = res.test == res.predict
analys = res.groupby(['diff', 'test']).count().predict

In [31]:
analys[0] / (analys[0] + analys[1])

test
0    0.259259
1    0.054545
2    0.066667
3    0.037500
4    0.142857
5    0.092593
6    0.014085
Name: predict, dtype: float64

Больше всего модель ошибалась в предсказании понедельника

In [32]:
file_name = '../data/model_regulirization.sav'
joblib.dump(best_model, file_name)

['../data/model_regulirization.sav']